# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**27/04/2020 - C8 S4**

## Tratamiento de datos y Exploración 

La el manejo de la información contenida en los datos es la razón principal de la construcción de modelos y esquemas de análisis sobre ellos. Tal información se ve afectada por la calidad de los datos, que en segunda instancia, determina el rendimiento de los modelos planteados. Esto hace que sea critico asegurar un preprocesado y una buena examinación de los datasets a trabajar.

El contenido de esta cátedra, se centra en las técnicas esenciales para el preprocesado de datos. 

### Análisis de datos exploratorio

El análisis exploratorio de los datos (EDA en inglés) consiste e utilizar técnicas de sumarización o agregación, con el fin de conocer la distribución de los datos, confirmar hipótesis y contrastar información. Existen muchas maneras de explorar los datos, por ejemplo, se pueden generan visualizaciones, descripciones del conjunto de datos, se pueden también generar agrupaciones y obtener patrones de tales agrupaciones. 

Un concepto recurrente en el análisis exploratorio de datos consiste en el *perfilamiento* de datos. Este hace referencia a la sumarización por medio de estadística descriptiva, aquí existe una variedad de herramientas que pueden ayudar a comprender mejor los datos disponibles. La meta del perfilamiento de datos consiste en generar respuestas y conocimiento en torno al fenómeno que los datos reflejan. En función de los perfiles generados, se puede tener una idea de la calidad del dataset con lo cual es posible decidir como transformar las variables a disposición. A continuación se da una guía a seguir al momento de explorar los datos

#### Perfilamiento Univariado

El punto inicial para comprender la naturaleza de una variable, pasa por caracterizar la forma de su distribución, un histograma permite obtener ideas sobre tal faceta.

**Ejemplo**

Se cargan las librerías iniciales

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats

Se carga el dataset de 'house pricing', este consiste en 80 variables (79 variables explicativas más una variable objetivo), describiendo aspectos fundamentales de hogares residenciales en la ciudad de Ames, Iowa. Este dataset está centrado en la regresión sobre el precio final de cada hogar. A continuación se procede a explorar tal dataset.

In [ ]:
# El conjunto a trabajar es el de entrenamiento
df = pd.read_csv('data/train.csv', index_col = 'Id')
df.head()

La cantidad de observaciones corresponde a 1460, por otra parte, posee 79 variables explicativas más un índice.

In [ ]:
df.shape

Se estudia el tipo de valor y cantidad de información faltante para cada columna

In [ ]:
df.info()

Observado lo anterior y la estructura de la base, se aprecia que los datos de tipo `object` hacen referencia a 'strings' o categorias del dataset. Se crea una lista con aquellas columnas.

In [ ]:
object_type_set = [col for col in df.columns if df[col].dtype == 'O']

Se observa la estructura del dataset en tales columnas, en este caso se decide transformarlas a formato 'str' para obtener visualizaciones sobre sus valores. El proceso de transformar los tipos de datos en un dataframe se conoce como *typecasting*. 

In [ ]:
# Se transoforman las columnas anteriores a 'str'
df = df.astype({col:'str' for col in object_type_set})

Sumado a lo anterior, se agregan niveles de multi indexado a las columnas para indicar si son del tipo numérico o categórico. 

In [ ]:
df.head()

In [ ]:
names = ['numeric', 'categorical']

# Se crea una lista con las columnas numericas
numeric = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal', 'GarageYrBlt',
    'MSSubClass','Fireplaces','SalePrice'
]

# Se crea una lista con las columnas categoricas
categorical = list(set(df.columns) - set(numeric))
''' 
Se generan mappings para el multi indexado del tipo 

[('numeric', col_if_numeric), ...,('categorical', col_if_categorical),...]
'''

mapping = [('numeric', col) for col in numeric]
mapping.extend([('categorical', col) for col in categorical])
'''
Se reordenan las columnas del dataframe para que coincidan con el esquema 
del multi indice
'''

df = df.reindex(columns=numeric + categorical)

Finalemente se asocia el multi indexado

In [ ]:
# Se reasignan las columnas
df.columns = pd.MultiIndex.from_tuples(mapping)

Como observación, se agrega que la columna 'MSSubClass' se clasifica como categórica pues representa un tipo de sector asociado a la propiedad. 

A continuación, se genera una visualización para entender la geometría de cada distribución, en el caso de las variables continuas, se calcula un estimado de la distribución por medio de `kernel density estimation`, este procedimiento consiste en elegir un tipo de función base (en este caso, una gaussiana con media en cada punto y de varianza constante) posteriormente, se calcula el promedio de las funciones base y se obtiene una función representante de la distribución denominada como 'density estimator', esto se hace por medio del método `sns.displot` de la librería `seaborn`. 

In [ ]:
# Grilla de subplots
fig, ax = plt.subplots(nrows=6, ncols=4, figsize=[17, 17])

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Numéricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), numeric):

    # Graficos para datos numericos
    sns.distplot(df[('numeric', col)], ax=axis, rug=True)
    axis.set_xlabel(col, fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.4, .4)
plt.subplots_adjust(wspace=w, hspace=h)

Para las variables categóricas, se genera un conteo de valores únicos. Dado que se buscan las distribuciones de forma visual, se elimina información referente a las escalas, que dada la cantidad de gráficos a obtener, solo entorpece el análisis.

In [ ]:
# Grilla de subplots
fig, ax = plt.subplots(nrows=10, ncols=6, figsize=[17, 17])

# Se remueven los ultimos 3 plots
list(map(lambda a : a.remove(), ax[-1,-3:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Categóricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), categorical):

    # Graficos para datos tipos str
    sns.countplot(df[('categorical',col)], ax=axis)
    axis.set_axis_off()
    axis.set_title(col, fontsize=15)
  
    
# Se ajusta el espaciado interno entre subplots
h, w = (.4, .1)
plt.subplots_adjust(wspace=w, hspace=h)

Al observar las distribuciones, es importante buscar si existe variabilidad dentro de estas, pues por lo general, una variable con un único valor casi seguro, no aporta información a la dinámica de los datos.

**Ejemplo**

Se observa la variable 'Heating' (categórica) y se compara con la variable de interés 'SalePrice'. Para ello se usa un gráfico de categórias tipo violín

In [ ]:
# Sirve para fija el tamaño de lasetiquetas del plot
fontdict = {'fontsize':20}

# Estrucutra de figura y axes
fig, ax = plt.subplots(2,1,figsize=[12,13])

# violin plot --> equivalente a catplot(kind = 'violin')

sns.violinplot(('categorical', 'OverallQual'),
            y=('numeric', 'SalePrice'),
            data=df,
            kind='violin',
            ax=ax[0])

sns.countplot(df[('categorical','OverallQual')], ax=ax[1])

ax[0].set_xlabel('OverallQual', fontdict)
ax[1].set_xlabel('OverallQual', fontdict)

ax[0].set_ylabel('SalePrice', fontdict)
ax[0].set_title('Violin plot OverallQuall vs SalePrice', fontdict)
ax[1].set_title('Frecuencias OverallQuall', fontdict)

h, w = (.3, .1)
plt.subplots_adjust(wspace=w, hspace=h)

Un gráfico de violín permite sumarizar y observar características de un dataset. Este se comporta como un gráfico de cajas (boxplot), mostrando la mediana, el rango intercuantílico IQR (percentil 75 - percentil 25, o Q3 - Q1) y el rango 1.5 intercuantílico (Q3 +- 1.5 IQR). Además de lo anterior, se suma una estimación de la densidad por kernel a cada lado. Esto quiere decir, que zonas con mayor densidad, se verán como 'montes' horizontales. 

En el caso de 'OverallQuall', se ve una clara relación entre los distintos niveles de está variable en contraste con difierentes distribuciones de 'SalePrice'. Junto con una distribción que presenta variabilidad, se podría considerar como una de interés. 


Por otra parte, analizando las gráficas univariadas, se puede observar que para 'LandSlope', se tiene poca variablidad y no genera diferencias en distribución para 'SalePrice' en ninguna de sus categorias. 

In [ ]:
# Sirve para fija el tamaño de lasetiquetas del plot
fontdict = {'fontsize':20}

# Estrucutra de figura y axes
fig, ax = plt.subplots(2,1,figsize=[12,13])

# violin plot --> equivalente a catplot(kind = 'violin')

sns.violinplot(('categorical', 'LandSlope'),
            y=('numeric', 'SalePrice'),
            data=df,
            kind='violin',
            ax=ax[0])

sns.countplot(df[('categorical','LandSlope')], ax=ax[1])

ax[0].set_xlabel('LandSlope', fontdict)
ax[1].set_xlabel('LandSlope', fontdict)

ax[0].set_ylabel('SalePrice', fontdict)
ax[0].set_title('Violin plot LandSlope vs SalePrice', fontdict)
ax[1].set_title('Frecuencias LandSlope', fontdict)

h, w = (.3, .1)
plt.subplots_adjust(wspace=w, hspace=h)

**Ejercicios**

1. Los gráficos generados anteriormente siguen exactamente el mismo patrón de generación, lo único que cambia es la columna a analizar. Esto es una mala práctica pues siempre se debe buscar reutilizar código o 'no repetirse' esto se conoce como principio DRY (don't repeat yourself). Construya una función que permita visualizar columnas categóricas del dataset y compararlas con 'SalePrice'. 

2. En función de las visualizaciones construidas, discuta que variables categóricas pueden ser de interés para predecir 'SalePrice'. Busque variabilidad y separación en la distribución de precios. ¿Qué ocurre si una variable categórica posee poca variablidad pero genera buenas separaciones en  'SalePrice'?

Para comparar las variables numéricas, se pueden utilizar gráficos de dispersión contra 'SalePrice'. En este caso, se buscan variabilidad en el histograma univariado y a la vez, se buscan relaciones funcionales (del tipo lineal, exponencial, cuadrático, etc..) con 'SalePrice'. 

**Ejemplo**

Debido a su distribución, se estudia la variable 'GrLivArea', en este caso se define una función para gráficar variables numéricas.

In [ ]:
def scatter_dists(col, df=df, h=.3, w=.1, fontdict={'fontsize': 20}, reg=True):
    ''' Recibe una columna numerica y genera una visualizacion comparativa.
    
    Genera una figura por sobre el dataframe HousePricing (por defecto), recibe 
    parametros extra como el espaciado entre subfigura.
    
    Args:
    ----------
    
    col: String
         El nombre de la columna numerica a visualizar
    
    h,w: float
        Espaciado entre subplot h -> vertical, w -> horizontal
    
    fontdict: dict
             Permite configurar las fuentes de los subplots
    reg: bool
         Permite graficar una regresion lineal sobre los datos (if True)
        
    Returns: None
        Se muestra una figura en pantalla    
    
    '''

    # Estrucutra de figura y axes
    fig, ax = plt.subplots(2, 1, figsize=[12, 13])

    # violin plot --> equivalente a catplot(kind = 'violin')

    if reg:
        sns.regplot(x=df[('numeric', col)],
                    y=df[('numeric', 'SalePrice')],
                    ax=ax[0])
        ax[0].set_title('Regplot plot {} vs SalePrice'.format(col), fontdict)
    else:
        sns.scatterplot(('numeric', col),
                        y=('numeric', 'SalePrice'),
                        data=df,
                        ax=ax[0])
        ax[0].set_title('Scatter plot {} vs SalePrice'.format(col), fontdict)

    
    # Distribucion univariada
    sns.distplot(df[('numeric', col)], ax=ax[1])

    ax[0].set_xlabel(col, fontdict)
    ax[1].set_xlabel(col, fontdict)

    ax[0].set_ylabel('SalePrice', fontdict)
    ax[1].set_title('Frecuencias {}'.format(col), fontdict)

    plt.subplots_adjust(wspace=w, hspace=h)
    

In [ ]:
scatter_dists('GrLivArea')

En este caso, se puede observar una distrbución univariada bien definida y un comportamiento lineal aunque ruidoso. Esto hace que 'GrLivArea' sea una variable de interés. De la misma manera, '1stFlrSF', parece reflejar las mismas buenas características. 



In [ ]:
scatter_dists('1stFlrSF') 

En el caso de 'TotalBsmtSF' se tiene

In [ ]:
scatter_dists('TotalBsmtSF')

Una relación menos lineal con un poco más de ruido pero una buena distribución en e dataset. Esta variable puede ser de interés pero esto se puede estudiar a posteriori. 

Finalmente para 'MasVnrArea', se tiene

In [ ]:
scatter_dists('MasVnrArea', reg = False)

Se aprecia una distribución altamente concentrada y poco relacionada con la variable a predecir, a priori, se puede considerar como una variable de poco interés en el análisis. 

**Ejercicio**

1. Estudie las siguientes proposiciones: 

    1.'OverallQual' y 'YearBuilt' parecen relacionadas con 'SalePrice'. 
    2. En el caso de 'OverallQual', esta relación es bastante débil.
    3. En el caso de 'YearBuilt', esta relación es bastante débil.
    4. Los gráficos de caja para 'OverallQual contra  'SalePrice' muestran cierta linealidad con respecto a 'SalePrice'.

2. Estudie la distribución univariada de 'SalePrice', a continuación ejecute el test K^2 de D’Agostino usando `normaltest` del módulo `stats` de SciPy. Compare para una significancia de 5%. ¿ Se puede tratar esta variable como distribuida de manera normal, tomando en cuenta su comportamiento estadístico?

3. Las distribuciones de 'TotalBsmtSF' y '1stFlrSF' parecen bastante similares, más aún sus relaciones con 'SalePrice' comparten una tendencia. Ejecute el [test de Kolmogorov-Smirnov ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html) por medio de `ks_2samp` para explorar la hipótesis:

 ```'TotalBsmtSF' y '1stFlrSF' vienen de la misma distribución```
 
4. Estudie algunos estadísticos de interés según el tipo de dato.
    
    1. Para las variables numéricas estudie promedios, desviaciones estándar y rangos intercuartílicos. Utilice los rangos calculados para tener una idea del porcentaje de valores fuera de tales rangos por columna. 
    
    2. Para variables catégoricas calcule frecuencias, proporciones y modas. Utilice lo anterior para obtener alguna idea de la variabilidad de los datos.
    

#### Perfilamiento Bivariado

Basándose en el perfilamiento anterior, es de utilidad observar relaciones entre variables de interés. Para esto se pueden emplear visualizaciones a pares. 

**Ejemplo**

Se selecciona un conjunto de variables de interés y se investigan sus relaciones bivariadas.

In [ ]:
# Se genera una función auxiliar

def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

Se selecciona un conjunto de variables a examinar

In [ ]:
interest = [
    'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    'FullBath', 'YearBuilt'
]

idxs = indexer(interest)

In [ ]:
df[idxs].head()

Se procede a observar el comportamiento bivariado de las columnas seleccionadas

In [ ]:
# Pone SalePrice al final de la lista
idxs.sort()
idxs.remove(('numeric', 'SalePrice'))
idxs.append(('numeric', 'SalePrice'))

In [ ]:
'''
Seaborn presenta problemas para multi indices en columnas, se 
procede a eliminar el nivel exterior y a obtener la visualización
correspondiente.
'''
data = df.reindex(idxs, axis=1).droplevel(0,axis=1)
sns.pairplot(data = data, diag_kind='kde')

La última fila de la visualización anterior entrega una idea de la relación entre 'SalePrice' y las demás variables de interés. Dentro de estas relaciones, se observan ciertos comportamientos lineales y en particular para 'OveralQuall' y 'YearBuilt' se observa cierta exponencialidad. Dentro de las interacciones entre variables, se observa que 'GrLivArea' y 'TotalBsmtSf' se comportan de manera similar contra 'OverllQuall', esperandose cierta tendencia creciente en ambos casos.

Los análsis iniciales basados en visualizaciones sirven para comprender a grandes rasgos la estructura del dataset. Este tipo de exploración debe ser acompañada de tests estadísticos como los vistos en los ejercicios anteriores. En el caso del perfilamiento bivariado se puede usar una técnica mixta, basada en el análisis de las correlaciones.

**Ejemplo**

Se construye una matriz de correlaciones y se visualiza para todo el dataset.

In [ ]:
corrmat = df.corr()

se muestran las dos variables más correlacionadas (positivamente) con 'SalePrice'.

In [ ]:
col = indexer(['SalePrice'])
corrmat[col].nlargest(3,col)

En cuanto a correlación negativa, no se ven relaciones lineales inversas de mayor fortaleza 

In [ ]:
corrmat[col].nsmallest(3,col)

Se procede a visualizar

In [ ]:
'''
Se inserta 'SalePrice' como primera fila x columna de la matriz de correlacion
'''

unsorted = list(corrmat.columns)
unsorted.remove(*col)
unsorted.insert(0, *col)

sortd = pd.MultiIndex.from_tuples(unsorted)
corrmat = corrmat.reindex(index = sortd, columns = sortd)
'''
Dado lo anterior, se ajusta el anchor de colores con maximo en .9
y -0.5, para tener una perspectiva entorno a los valores maximos 
de correlacion (negativa y positiva)
'''

fig, ax = plt.subplots(figsize=[16, 14])

sns.heatmap(corrmat, vmin=-.5, vmax=.9, linewidths=.01)

Según el esquema de valores, se buscan los puntos más claros y más oscuros fuera de la diagonal. En primera instancia, las variables  'TotalBsmtSF' y '1stFlrSF' parece bastante correlacionadas, lo mismo ocurre con la variables 'GarageCars' y 'GarageArea', esto puede indicar multicolinearidad que implica información duplicada o relacionada de manera trivial en el dataset. 

Las correlaciones con 'SalePrice' deben ser analizadas con más detenimiento, aquí se ve que  'GrLivArea', 'TotalBsmtSF', y 'OverallQual' juegan un papel preponderante.

**Ejercicio**

1. Obtenga las 15 correlaciones más altas (positiva o negativa) con 'SalePrice'. Reindexe la matriz de correlaciones, de manera tal que contenga 1's en la diagonal y 'SalePrice' sea la primera fila - columna. 

2. Muestre los coeficiente de correlación dentro de cada casilla del gráfico de correlaciones. Utilice esa información en conjunción con el perfilamiento univariado para filtrar variables de interés.

Las correlaciones pueden ser interpretadas con datos mixtos pero se recomienda analizar sus valores cuando se trabaja con valores continuos (comparación variable continua vs continua). Para analizar valores categóricos (categórico vs categórico) existen herramientas especializadas una de ellas es por medio de tablas de dos tratamientos o de contingencia (2 way tables). 

**Ejemplo**

Se construye una tabla para analizar 'OverallQual' vs 'GarageCars'

In [ ]:
to_compare =['OverallQual','GarageCars']
data_cat = df['categorical']

kwargs = {'index': data_cat[to_compare[0]], 'columns': data_cat[to_compare[1]]}

# Se construye la tabla
tabla = pd.crosstab(**kwargs, margins=True, margins_name='Total')
tabla

En el caso anterior, la función `pd.crosstab(**kwargs, margins=True, margins_name='Total')` es equivalente a

```python
data_cat.pivot_table(**kwargs, values = 'OverallQual',aggfunc='count', fill_value=0)
```
y permite calcular el numero de ocurrencias de una variable para cada una de sus categorías en comparación con los valores de otra variable. Se añaden los totales como margenes de la tabla. En este caso, podemos deducir las interacciones entre las variables, de manera similar como actúa la correlación en variables continuas. 

Para el caso de  'OverallQual' y 'GarageCars' vemos que tienden a acumularse dentro de una rango reducido, se puede concluir que a medida que 'OverallQual' crece entre 4 y 6, aparece un aumento considerable en la categoría 'GarageCars' hasta que esta última llega al valor 2, valores superiores paracieran ser independientes de 'OverallQual'. 

**Ejercicios**

1. Compare variables categóricas usando este método, ¿ se puede encontrar alguna relación entre categórias?

2. Es posible aplicar este método para comparar variables categóricas y continuas, para esto se necesita categorizar la variable continua objetivo. Categoríce la variable 'SalePrice' en 5 tramos y compare con 'OverallQual' ¿Se observa alguna tendencia?


Otra forma de comparar variables categóricas es por medio de un test $\chi^2$. Este permite obtener un indicador de significancia estadística entre variables. Se basa en una tabla de contingencia y proporciona la probabilidad de que dos variables categóricas sean independientes basádandose en el estádistico $\chi^2$, entrega también un arreglo con frecuencias esperadas.

In [ ]:
from scipy.stats import chi2_contingency

# Se debe trabajar la tabla sin margenes
tabla = pd.crosstab(**kwargs, margins=False)

chi2, p, dof, ex =chi2_contingency(tabla)

La tabla de frecuencias esperadas se puede interpretar de la siguiente forma:

In [ ]:
expected_freq = pd.DataFrame(ex, index=range(1,11))
expected_freq.index.name = 'OverallQual'
expected_freq.columns.name = 'GarageCars'
expected_freq

Aquí, la frecuencia esperada para la la categoría 1 'OverallQual' de estar en la categoría 0 de 'GarageCars' es 0.11. Se puede decir que esta configuración es muy poco probable en comparación a otras como pertenecer a la categoría 6 de 'OverallQual' y 2 de 'GarageCars'. Este tipo de tablas permite clasificar las relaciones entre variables categóricas y obtener *insights* sobre las dinámicas que el dataset refleja. 

El valor $p$ entregado por el cáculo corresponde a:

In [ ]:
p 

Si para este test llamamos $\alpha$ al valor de significancia, se puede resumir:

1. Si $p > \alpha$ no hay evidencia para rechazar la hipótesis nula por lo que se pueden considerar independientes.

2. Si $p \leq \alpha$ hay evidencia para rechazar la hipótesis nula por lo que se puede decir que existe una dependencia estadística entre las variables. 


Para una significancia del 5% , hay evidencia para rechazar la hipótesis de independencia entre 'OverallQual' y 'GarageCars', luego puede existir un factor latente que las relaciona (¿será 'SalePrice'?)

In [ ]:
p <= 0.05

Como consideración general, para que este test sea consistente estadísticamente, se deben observar frecuencias (esperadas y observadas) mayores a 5. 

**Ejercicio**

1. Utilice el gráfico de correlaciones para escojer dos variables categóricas de interés. Verifique si existen relaciones estadísticas entre ellas. 

Finalmente, para comparar variables numéricas y categóricas, es posible utilizar técnicas especializadas como lo son los tests Z y T. Estos tests se utilizan de manera simultanea con gráficos de caja (o violín), donde cada caja representa una categoría. 

Tanto el test Z como el T permiten verificar si las medias de dos grupos son estadísticamente diferentes entre si. aquí, el estadístico Z se define por

\begin{equation}z=\frac{\left|\bar{x}_{1}-\bar{x}_{2}\right|}{\sqrt{\frac{s_{1}^{2}}{n_{1}}+\frac{s_{2}^{2}}{n_{2}}}}\end{equation}

Si su probabilidad asociada es pequeña, entonces **la diferencia de las medias** es significativa. 

Por otra parte, el estadístico T es más robusto a tamaños de observaciones pequeños (menores que 30 por ejemplo), este viene dado por 

\begin{equation}
t=\frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{S^{2}\left(\frac{1}{N_{1}}+\frac{1}{N_{2}}\right)}}
\end{equation}

Donde 

\begin{equation}
S^{2}=\frac{\left(N_{1}-1\right) S_{1}^{2}+\left(N_{2}-1\right) S_{2}^{2}}{N_{1}+N_{2}-2}
\end{equation}

Aquí , $\bar{X}_{1}, \bar{X}_{2}$ son las medias, $S_{1}^{2}, S_{2}^{2}$ varianzas y $N_1$ , $N_2$ los totales de cada grupo a testear. 

**Ejercicios**

1. Utilice el test de independencia $t$ (o 2 - sample $t$-test) para comparar 2 variables continuas de interés. [*Hint*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

2. Observe que el caso categórico vs continuo, cada categoría representa un grupo de valores continuos asociados. Por ejemplo, si la variable categórica `A` tiene las categorías `c_1` y `c_2`, al compararla con la variable continua `B`, es necesario agrupar los valores de `B` para `c_1` y para `c_2` para luego estudiar su independencia. Utilice el test anterior para medir independencias de grupos entre una variable categórica vs 'SalePrice'. **Obs**: La variable categórica debe ser bivariada.

3. Utilice el test Z con las variables anteriores y compare. ¿Qué restricciones extra posee este test?

Finalmente, se puede hacer uso de un test F o ANOVA. Este test permite comparar más de una media al mismo tiempo, una manera simple de aplicar este test consiste en método conocido como **one way ANOVA**, aquí, se testea si más de 2 grupos son similares basados en sus medias. En este caso, la hipótesis nula es 

`No hay diferencia significativa entre los grupos`

**Ejemplo**

Se selecciona la variable 'GarageCars' y se compara con 'SalePrice'. 

In [ ]:
idx = indexer(['SalePrice','GarageCars'])
grouped = df[idx].groupby(idx[1])

En la variable 'GarageCars' se distinguen 5 categorías

In [ ]:
len(df[idx[1]].unique())

a partir de la agrupación anterior, se forman entonces 5 grupos de valores para 'SalePrice'.

In [ ]:
len(grouped.groups)

Se obtienen los grupos

In [ ]:
total_groups = len(grouped.groups)
groups = [grouped.get_group(i) for i in range(total_groups)]

Se muestra el grupo correspondiente a la categoría 0

In [ ]:
groups[0].head()

Se limpia el formato de cada grupo

In [ ]:
def group_cleaner(group):
    ''' Limpia un grupo.
    Reconoce la categoria del grupo, en la posicion [:,1], 
    guarda ese nombre y elimina la columna de categoria, 
    posteriormente renombra la columna.
    
    Args:
    ----------
    
    group: pandas Groupby object
          Recibe una agrupacion para categorias
          
    Returns:
    ----------
        pandas Grppuby object
        Entrega el grupo ordenado.
    '''
    group_0 = group.copy()
    name = group_0.iloc[0,1]
    group_0.drop(indexer(['GarageCars']), axis=1, inplace=True)
    group_0.columns  = ('cat_{}'.format(name),)
    
    return group_0

se procede a limpiar

In [ ]:
groups_to_test = list(map(group_cleaner, groups))

Se muestra el grupo correspondiente a la categoria 0 post limpieza

In [ ]:
groups_to_test[0].head()

Se procede a testear

In [ ]:
from scipy.stats import f_oneway

F,p = f_oneway(*groups_to_test)

print('Estadistico F:',F)
print('p valor :', p)

probando para una significancia del 5% se tiene hay evidencia para rechazar la hipótesis nula y por tanto hay una diferencia significativa entre los grupos. 

In [ ]:
alpha = 0.05
p <= alpha

**Ejercicio**

1. Compruebe el resultado del test ANOVA anterior con un analísis visual por medio de gráficos de violín.

### El problema de los datos faltantes

Los métodos estándar de manejo de datos han sido desarrollados para para analizar arreglos tabulares. Por lo general las filas de tal arreglo representan observaciones y las columnas sus características asociadas. Cada entrada en este arreglo puede ser modelada como un número, siendo este ligados a un proceso subyacente continuo o discreto. 

Como definición, se considerará como **valor faltante** a aquellas entradas con características no observadas, cuyo potencial informativo es significativo en el análisis a realizar. Al aplicar esta definición, tiene sentido considerar métodos de llenado (inputación) para completar la información perdida. 

Para dar un contexto teórico al problema de valores faltantes, se define la matriz de datos $Y=\left(y_{i j}\right)$, la cual representa un arreglo rectangular de datos. Para cada elemento de esa matriz, se asocia una variable indicadora, que da lugar a una *matriz indicadora de información faltante* definida por  
$R=\left(r_{i j}\right)$. 

Por simplicidad, se puede asumir que las filas $\left(y_{i}, r_{i}\right)$ son i.i.d sobre $i$. Es posible así, modelar la existencia de un proceso (o fenómeno) que causa la perdida de información sobre $Y$ por medio de una distribución condicional de $R$  dado $Y$, denotada por $\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)$, donde $\phi$ denota un conjunto de parámetros que modelan el proceso de perdida de información e $Y_{obs}$, $Y_{mis}$, denotan aquellas entradas de la matriz de datos $Y$ con información observada y faltante respectivamente. Si para tal proceso no se aprecia una relación con los valores faltantes en $Y$, es decir, 

\begin{equation}
\tag{eq:1}
\label{eq:1}
\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)= 
\mathcal{P} \left(R \mid \phi \right)
\end{equation}

Se dice entonces que el mecanismo de perdida de información es del tipo *información faltante completamente aleatoria* o MCAR (missing completely at random). Por otra parte, cuando la probabilidad de la información faltante en $Y$ se relaciona con variables observadas, así, $R$ depende de $Y_{obs}$ pero no de $Y_{mis}$, por lo que la distribución pasa a ser


\begin{equation}
\tag{eq:2}
\label{eq:2}
\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)=
\mathcal{P} \left(R \mid Y_{obs}, \phi \right)
\end{equation}

Entonces, el proceso de perdida de información se denomina como, *información faltante aleatoria* MAR (missing at random). El proceso es llamado *información faltante no aleatoria* MNAR, si la distribución de $R$ depende de las componentes faltantes de $Y_{mis}$, es decir, la ecuación inicial $\ref{eq:1}$ no se cumple para algunas filas de $Y$ y algunos valores de las componentes faltantes.

En los métodos discutidos en esta cátedra $R$ e $Y$ serán modelados por medio de distribuciones conjuntas, es decir, son tratadas como variables aleatorias. 

**Ejemplo**

La estructura de datos con información faltante más simple, ocurre en el caso univariado. Acá, $Y$ y $R$ son vectores, luego 

\begin{equation}
\mathcal{P}(Y=y, R=m | \theta, \phi)=\prod_{i=1}^{n} f_{Y}\left(y_{i} | \theta\right) \prod_{i=1}^{n} f_{R \mid Y} \left(r_{i} | y_{i}, \phi\right)
\end{equation}

donde $f_{Y}\left(y_{i} | \theta\right)$ denota la densidad de la componente $i$-sima de $Y$, $y_{i}$, parametrizada por $\theta$, y $ f_{R \mid Y} \left(r_{i} | y_{i}, \phi\right)$ es la densidad de una variable aleatoria Bernoulli para el indicador binario $r_{i}$, con probabilidad $\mathcal{Pr}\left(r_{i}=1 | y_{i}, \phi\right)$ para $y_{i}$ valor faltante. Si el proceso de perdida de información es independiente de $Y$, es decir, $\mathcal{Pr}\left(r_{i} = 1 | y_{i}, \phi\right)=\phi$ , constante que no depende de $y_{i}$, entonces el proceso de perdida de información es MCAR. Si tal mecanismo depende de $y_{i}$, entonces es MNAR pues pasa a depender de los valores perdidios de $y_{i}$.

**Ejercicio**

Si se supone que $Y$ es una variable $n$ dimensional con posibles valores faltantes, $R$ es el indicador de pérdida de información para $Y$ y $X$ es una variable  $n$ dimensional relacionada al mismo dataset pero con valores completamente observados. En este contexto, si el dataset consiste de $n$ observaciones, donde para $r < n$  fijo, se tiene que $i=1, \ldots, r$ $X_i$ e $Y_i$ son observados, mientras que para $j = r+1, \ldots, n$ se tiene $X_j$ observado pero $Y_j$ faltante. 

1. Si para $i = 1, \ldots, n$ se asume que $y_i$ es independiente de $r_i$ dado $x_i$. Aplique el supuesto MAR sobre este conjunto de datos y deduzca una expresión para $\mathcal{P}(R \mid X, Y , \phi)$ que no dependa de $Y$. 

2. Utilice la expresión anterior para deducir que $R$ e $Y$ son independientes dado $X$. 

3. Utilice lo anterior para deducir que la distribución condicional de $Y$ dado $X$ y $R$ no depende de $R$. 

4. Deduzca que la distribución condicional de $Y$ dado $X$ puede ser estimada para componentes con $Y$ observado ($r_i = 0$) para luego ser utilizada para predecir valores faltantes de $Y$ ($r_i = 1$). 




 

Para comprender el significado de lo anterior, considere el siguiente ejemplo

**Ejemplo**

Se realiza un experimento en cual se examinan ciertas variables relacionadas a la salud, dentro de estas variables, se consideran algunos comportamientos como *beber alcohol*, *fumar*, *consumo de drogas* y *actividad sexual*. El experimento se realiza sobre una población menor de 18 años, luego, por regulaciones gubernamentales, no se pueden realizar preguntas sobre actividad sexual a menores de 14 años. Observe

1. La variable asociada al comportamiento sexual cumple la hipótesis MAR, pues en efecto, al observar la variable de edad (variable observada $Y_{obs}$), se puede comprender la falta de información subyacente.

2. Por otra parte, suponga que la variable de edad y comportamiento sexual están altamente correlacionadas. Asuma además que el estudio existe un valore de *índice de salud* asociado a cada participante. Los investigadores del experimento deciden utilizar regresión lineal sobre el valor del *índice de salud* para estudiar el peso de cada variable en el modelo. Al hacer esto, eliminan la variable de edad por ser redundante. 
Si los investigadores relacionan los valores del *índice de salud* con la probabilidad de que la variable sexual tenga valores faltantes, se está en la hipótesis MNAR, pues la variable edad pasa a ser no observada. 